# Modelling & Simulation — Emergency Revision Short Course (Easy English)

**Goal:** revise fast for the exam (queues + random numbers + Monte Carlo + automata).
**English level:** simple sentences, small steps, many examples.

## How to use

- Read the rule.
- Do 1–2 examples.
- Try the mini-exercise.
- Check the solution.

---

**Symbols (we use them a lot):**

- $\lambda$ (lambda): arrival rate (customers per hour)
- $\mu$ (mu): service rate (customers per hour)
- $c$: number of servers
- $\rho$ (rho): utilization (traffic intensity)
- $L$: average number in system
- $W$: average time in system
- $L_q$: average number waiting
- $W_q$: average waiting time


# Topic 1 — Queues (Files d’attente)

## 1) Kendall notation

We write: **A/B/c/K**

- **A**: arrivals (often **M** = Poisson / exponential inter-arrival)
- **B**: service (often **M** = exponential, **D** = deterministic)
- **c**: number of servers
- **K**: capacity (if missing, it means $\infty$)

> In exam, the most common is **M/M/1** and **M/M/c**.

---

## 2) M/M/1 (one server) — the must-know model

### Stability (very important)

The system is stable **only if**:
$$\rho = \frac{\lambda}{\mu} < 1$$
If $\rho \ge 1$, the queue grows and results are “infinite”.

### Formulas to memorize (M/M/1)

$$\rho = \frac{\lambda}{\mu}$$
$$P_0 = 1-\rho$$
$$L = \frac{\rho}{1-\rho}$$
$$W = \frac{1}{\mu-\lambda}$$
$$L_q = \frac{\rho^2}{1-\rho}$$
$$W_q = \frac{\rho}{\mu-\lambda}$$

### Little’s Law (cheat code)

$$L = \lambda W \quad\text{and}\quad L_q = \lambda W_q$$
If you know $L$ you can get $W = L/\lambda$ (and same for $q$).


## 3) M/M/1 — Worked examples (with steps)

### Example 1 (Bank)

Customers arrive: $\lambda = 10$ per hour. One teller serves: $\mu = 12$ per hour.

1. Compute $\rho = \lambda/\mu = 10/12 = 0.833$ (stable)
2. $W = 1/(\mu-\lambda)=1/(12-10)=0.5$ hours = **30 minutes**
3. $L = \lambda W = 10 * 0.5 = 5$ customers (Little’s Law)
4. $W_q = \rho/(\mu-\lambda) = 0.833/2 = 0.4167$ hours = **25 min**
5. $L_q = \lambda W_q = 10 * 0.4167 = 4.167$ customers

### Example 2 (Repairman)

Arrivals: $\lambda = 4$ per hour. Service time is 10 minutes on average.

- 10 minutes = 1/6 hour, so $\mu = 6$ per hour.
- $\rho = 4/6 = 0.667$ (stable)
- $W = 1/(6-4) = 0.5$ hour = 30 min
- $L = \lambda W = 4 * 0.5 = 2$

### Example 3 (Check stability quickly)

If $\lambda = 15$ per hour and $\mu = 12$ per hour:

- $\rho = 15/12 = 1.25$
- Not stable → do **not** use M/M/1 formulas (results blow up).

## Mini-exercise A (try first)

Given $\lambda=8$/hour and $\mu=10$/hour: find $\rho$, $W$, $L$, $W_q$, $L_q$.

### Solution A

- $\rho = 8/10 = 0.8$
- $W = 1/(10-8)=0.5$ hour = 30 min
- $L = \lambda W = 8*0.5 = 4$
- $W_q = \rho/(\mu-\lambda)=0.8/2=0.4$ hour = 24 min
- $L_q = \lambda W_q = 8*0.4=3.2$


In [3]:
# Helper functions for M/M/1 (quick exam checks)
from dataclasses import dataclass
@dataclass
class MM1Result:
    rho: float
    P0: float
    L: float
    W: float
    Lq: float
    Wq: float

def mm1(lambda_rate: float, mu_rate: float) -> MM1Result:
    if mu_rate <= 0 or lambda_rate < 0:
        raise ValueError("Rates must be: lambda >= 0 and mu > 0")
    rho = lambda_rate / mu_rate
    if rho >= 1:
        raise ValueError(f"Unstable system: rho={rho:.3f} >= 1")
    P0 = 1 - rho
    L = rho / (1 - rho)
    W = 1 / (mu_rate - lambda_rate)
    Lq = (rho**2) / (1 - rho)
    Wq = rho / (mu_rate - lambda_rate)
    return MM1Result(rho=rho, P0=P0, L=L, W=W, Lq=Lq, Wq=Wq)

print(mm1(10, 12))

MM1Result(rho=0.8333333333333334, P0=0.16666666666666663, L=5.000000000000002, W=0.5, Lq=4.166666666666668, Wq=0.4166666666666667)


## 4) M/M/c (multi-server) — what to focus on

### Stability

For **M/M/c** the stability condition is:
$$\rho = \frac{\lambda}{c\mu} < 1$$

- If stable: system works (finite averages).
- If not stable: queue explodes.

### Exam tip

Often they do **not** ask the full complicated formulas (Erlang-C).
So focus on:

- stability
- Little’s Law
- common sense: more servers → less waiting

### Example (stability only)

Arrivals $\lambda=30$/hour. Each server $\mu=12$/hour.

- If $c=2$: $\rho = 30/(2*12)=30/24=1.25$ not stable
- If $c=3$: $\rho = 30/(3*12)=30/36=0.833$ stable


# Topic 2 — Random number generation (Génération aléatoire)

We often start from **Uniform** random numbers $U\sim\mathcal U(0,1)$.
Then we transform $U$ to get other distributions.

---

## 1) Linear Congruential Generator (LCG)

We generate integers $X_n$ with:
$$X_{n+1} = (aX_n + c) \bmod m$$
Then we create a uniform number:
$$U_{n+1} = \frac{X_{n+1}}{m}$$

- $X_0$ is the **seed** (start value).

### Example (small LCG)

Let $m=16$, $a=5$, $c=1$, seed $X_0=7$.
Compute 3 steps:

- $X_1=(5*7+1)\bmod 16 = 36\bmod 16 = 4$, so $U_1=4/16=0.25$
- $X_2=(5*4+1)\bmod 16 = 21\bmod 16 = 5$, so $U_2=0.3125$
- $X_3=(5*5+1)\bmod 16 = 26\bmod 16 = 10$, so $U_3=0.625$


In [4]:
# LCG in Python (educational, not cryptography)
def lcg(seed: int, a: int, c: int, m: int, n: int):
    x = seed
    for _ in range(n):
        x = (a * x + c) % m
        yield x / m

u = list(lcg(seed=7, a=5, c=1, m=16, n=10))
u

[0.25, 0.3125, 0.625, 0.1875, 0.0, 0.0625, 0.375, 0.9375, 0.75, 0.8125]

## 2) Inverse transform method (Méthode d’inversion)

Idea: If we know the CDF $F_X(x)$, we can generate $X$ from $U\sim\mathcal U(0,1)$ by:
$$U = F_X(X) \Rightarrow X = F_X^{-1}(U)$$

### (a) Exponential distribution $\text{Exp}(\lambda)$

CDF: $F(x)=1-e^{-\lambda x}$ for $x\ge 0$
Inverse:
$$X = -\frac{1}{\lambda}\ln(1-U)$$
Often we also use $X = -\frac{1}{\lambda}\ln(U)$ (same distribution).

Worked example
Let $\lambda=2$ and $U=0.20$:
$$X = -\frac{1}{2}\ln(1-0.20)= -0.5\ln(0.8) \approx 0.1116$$

### (b) Uniform distribution $\mathcal U(a,b)$

$$X = a + (b-a)U$$
Example: $a=5$, $b=9$, $U=0.3$ → $X=5+4*0.3=6.2$

### (c) Bernoulli($p$)

If $U < p$ then $X=1$, else $X=0$.
Example: $p=0.7$, $U=0.65$ → $X=1$ ; $U=0.9$ → $X=0$


In [5]:
import math, random

def inv_exp(lam: float, u: float | None = None) -> float:
    if u is None:
        u = random.random()
    # use -ln(1-u)/lam to avoid ln(0) if u=0
    return -math.log(1 - u) / lam

def inv_uniform(a: float, b: float, u: float | None = None) -> float:
    if u is None:
        u = random.random()
    return a + (b - a) * u

def inv_bernoulli(p: float, u: float | None = None) -> int:
    if u is None:
        u = random.random()
    return 1 if u < p else 0

print("Exp example (lam=2, u=0.2):", inv_exp(2, 0.2))
print("Uniform example (5,9,u=0.3):", inv_uniform(5, 9, 0.3))
print("Bernoulli example (p=0.7,u=0.65):", inv_bernoulli(0.7, 0.65))

Exp example (lam=2, u=0.2): 0.11157177565710485
Uniform example (5,9,u=0.3): 6.2
Bernoulli example (p=0.7,u=0.65): 1


## 3) Rejection method (Méthode de rejet)

Use this when $F^{-1}$ is hard.
We want a target density $f(x)$.
We choose an easy density $g(x)$ and a constant $c$ such that:
$$f(x) \le c\,g(x) \text{ for all } x$$
Algorithm:

1. Sample $Y \sim g$
2. Sample $U\sim\mathcal U(0,1)$
3. Accept $Y$ as $X$ if $U \le \frac{f(Y)}{c g(Y)}$

Simple example (toy)
Target: $f(x)=2x$ on $[0,1]$ (a triangle).
Choose $g(x)=1$ on $[0,1]$ (uniform).
We need $c$ so that $2x \le c*1$. Max is 2, so choose $c=2$.
Accept if $U \le (2Y)/2 = Y$.
So: sample $Y\sim U(0,1)$, sample $U$, accept if $U\le Y$.


In [6]:
import numpy as np

def rejection_triangle(n: int, rng: np.random.Generator | None = None) -> np.ndarray:
    rng = rng or np.random.default_rng()
    samples = []
    while len(samples) < n:
        y = rng.random()       # Y ~ Uniform(0,1)
        u = rng.random()       # U ~ Uniform(0,1)
        if u <= y:             # accept with prob = y
            samples.append(y)
    return np.array(samples)

tri = rejection_triangle(10000)
tri.mean(), tri.min(), tri.max()

(np.float64(0.6693634987441736),
 np.float64(0.013727679258776382),
 np.float64(0.9998870186552195))

## 4) Box–Muller method (Normal $\mathcal N(0,1)$)

To generate a normal variable from two uniforms $U_1,U_2\sim\mathcal U(0,1)$:
$$Z_1 = \sqrt{-2\ln(U_1)}\cos(2\pi U_2)$$
$$Z_2 = \sqrt{-2\ln(U_1)}\sin(2\pi U_2)$$
Both $Z_1$ and $Z_2$ are $\mathcal N(0,1)$.

If you need $\mathcal N(\mu,\sigma^2)$:
$$X = \mu + \sigma Z$$


In [7]:
import numpy as np

def box_muller(n: int, rng: np.random.Generator | None = None) -> np.ndarray:
    rng = rng or np.random.default_rng()
    u1 = rng.random(n)
    u2 = rng.random(n)
    r = np.sqrt(-2.0 * np.log(u1))
    z = r * np.cos(2.0 * np.pi * u2)
    return z

z = box_muller(20000)
float(z.mean()), float(z.std(ddof=0))

(0.0025382164468538333, 1.003686587034201)